# W207 Final Project
- Looks like the competition is predicting the log error, not actual home price. If we want to do home price, we may have to pull out own data


Experiment with the Zillow API to grab more recent data? https://www.zillow.com/howto/api/PropertyDetailsAPIOverview.htm
- GetDeepComps - Has last sale price I think
- GetDeepSearch-Results - Also has last sale price
- GetUpdatedPropertyDetails - Has most detailed property information

Other info like the property tax might be useful

In [8]:
"""
All the packages we'll need for the various algorithms we can use
"""

# Global imports
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline

np.random.seed(0)

"""
Decision Trees
Source: Week 4 Notebook


"""

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# For producing decision tree diagrams.
from IPython.core.display import Image, display
from sklearn.externals.six import StringIO
import pydot

"""
Linear Regression
Source: Week 6 Notebook


"""

import seaborn as sns
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

np.set_printoptions(precision=4, suppress=True)

"""
Neural Network
Source: Week 7 Notebook and Week 10 notebook

- Jason recommends we use Keras, but I have included the Week 7 imports as well


from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

import theano 
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
print(theano.config.device) # We're using CPUs (for now)
print(theano.config.floatX) # Should be 64 bit for CPUs

"""

import keras
keras.__version__

"""
SVM
Source: Week 10 Notebook

from sklearn.linear_model import LinearRegression
"""

from scipy import stats
import pylab as pl

sns.set()

In [17]:
"""
Import the Zillow Home Value Prediction data from the Kaggle Competition

Try to run this a few times as possible, the data set takes a little while to import
"""

# Import the data dictionary for a summary of the features we have
data_dictionary = pd.read_csv('..\Data\zillow_data_dictionary\zillow_data_dictionary.csv', sep=',')

# Import the 2016 data
# Will need to address NaN
# Split into train, test, dev sets
data_2016 = pd.read_csv('..\Data\properties_2016\properties_2016.csv', sep=',')

# Shuffle the data
np_data_2016 = np.asarray(data_2016)
shuffle = np.random.permutation(np.arange(np_data_2016.shape[0]))
np_data_2016 = np_data_2016[shuffle]

# Check the shape
print('data shape:', np_data_2016.shape)

c:\program files\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


data shape: (2985217, 58)


In [21]:
train_2016 = pd.read_csv('..\Data\train_2016_v2\train_2016_v2.csv', sep=",")

train_2016.describe()

FileNotFoundError: File b'..\\Data\train_2016_v2\train_2016_v2.csv' does not exist

In [12]:
# Set some variables for test, dev, and training data.
train_data_2016 = np_data_2016[:1000000]
test_data_2016 = np_data_2016[1000000:2000000]
dev_data_2016 = np_data_2016[2000000:]
mini_train_data_2016 = np_data_2016[:10000]

print(data_dictionary)

                           Feature  \
0          'airconditioningtypeid'   
1       'architecturalstyletypeid'   
2                   'basementsqft'   
3                    'bathroomcnt'   
4                     'bedroomcnt'   
5          'buildingqualitytypeid'   
6            'buildingclasstypeid'   
7              'calculatedbathnbr'   
8                     'decktypeid'   
9            'threequarterbathnbr'   
10      'finishedfloor1squarefeet'   
11  'calculatedfinishedsquarefeet'   
12           'finishedsquarefeet6'   
13          'finishedsquarefeet12'   
14          'finishedsquarefeet13'   
15          'finishedsquarefeet15'   
16          'finishedsquarefeet50'   
17                          'fips'   
18                  'fireplacecnt'   
19                 'fireplaceflag'   
20                   'fullbathcnt'   
21                  'garagecarcnt'   
22               'garagetotalsqft'   
23                'hashottuborspa'   
24         'heatingorsystemtypeid'   
25          

In [16]:
# Summaries of variables
data_2016.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,2.985217e+06,811519.000000,6061.000000,1628.000000,2.973755e+06,2.973767e+06,12629.000000,1.938488e+06,2.856305e+06,17096.0,...,2647.000000,2.925289e+06,682069.000000,2.930235e+06,2.942667e+06,2.973778e+06,2.917484e+06,2.953967e+06,56464.000000,2.910091e+06
mean,1.332586e+07,1.931166,7.202607,646.883292,2.209143e+00,3.088949e+00,3.725948,5.784787e+00,2.299263e+00,66.0,...,278.296562,1.964262e+03,1.401464,1.708836e+05,4.204790e+05,2.014999e+03,2.524780e+05,5.377607e+03,13.892409,6.048431e+13
std,7.909966e+06,3.148587,2.436290,538.793473,1.077754e+00,1.275859e+00,0.501700,1.805352e+00,1.000736e+00,0.0,...,369.731508,2.344132e+01,0.539076,4.020683e+05,7.263467e+05,3.683161e-02,4.450132e+05,9.183107e+03,2.581006,3.249035e+11
min,1.071172e+07,1.000000,2.000000,20.000000,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,66.0,...,10.000000,1.801000e+03,1.000000,1.000000e+00,1.000000e+00,2.000000e+03,1.000000e+00,1.340000e+00,0.000000,-1.000000e+00
25%,1.164371e+07,1.000000,7.000000,272.000000,2.000000e+00,2.000000e+00,3.000000,4.000000e+00,2.000000e+00,66.0,...,96.000000,1.950000e+03,1.000000,7.480000e+04,1.796750e+05,2.015000e+03,7.483600e+04,2.461070e+03,14.000000,6.037400e+13
50%,1.254509e+07,1.000000,7.000000,534.000000,2.000000e+00,3.000000e+00,4.000000,7.000000e+00,2.000000e+00,66.0,...,168.000000,1.963000e+03,1.000000,1.225900e+05,3.060860e+05,2.015000e+03,1.670420e+05,3.991780e+03,14.000000,6.037572e+13
75%,1.409712e+07,1.000000,7.000000,847.250000,3.000000e+00,4.000000e+00,4.000000,7.000000e+00,3.000000e+00,66.0,...,320.000000,1.981000e+03,2.000000,1.968890e+05,4.880000e+05,2.015000e+03,3.069180e+05,6.201005e+03,15.000000,6.059042e+13
max,1.696019e+08,13.000000,27.000000,8516.000000,2.000000e+01,2.000000e+01,5.000000,1.200000e+01,2.000000e+01,66.0,...,6141.000000,2.015000e+03,41.000000,2.514860e+08,2.827860e+08,2.016000e+03,9.024622e+07,3.458861e+06,99.000000,4.830301e+14


In [6]:
linr =  LinearRegression()

In [29]:
import os
cwd = os.getcwd()

print(cwd)

E:\_FRESH START\MIDS\W207\w207_swift_larance\Project Notebooks
